In [ ]:
import pandas as pd
from column_names_groups import *
from mfyfunctions import *
from analysis_functions import *

In [ ]:
pd.set_option('display.float_format', '{:.1f}'.format)
pd.set_option('display.max_colwidth', None)
# noinspection PyTypeChecker
threshold_date = pd.Timestamp('2022-09-30')

In [ ]:
# intake0 = pd.read_csv(r'011/datasets/0001_ps_nt_092310.csv')
# intake0[intake_date_columns] = intake0[intake_date_columns].apply(pd.to_datetime)
# intake0[intake_numerical_variables] = intake0[intake_numerical_variables].astype('Int64')
# intake0 = intake0[(intake0[intake_date_columns] > threshold_date).any(axis=1)]

In [ ]:
# Exclude not completed intake assessments 
# intake0 = intake0[intake0['nt'] != -1]
# intake0['nt1'].count() + intake0['nt2'].count() + intake0['nt3'].count() + intake0['ntre'].count()

In [ ]:
# intake0

In [ ]:
# to calculate intake assessment occurrences, I will count only the first intake session and drop the 2nd and 3rd sessions.
# intake = intake.drop(['nt2', 'nt3'], axis=1)
# intake

In [ ]:
# melt_intake = melt_and_categorize_dates(
#     input_df=intake,
#     date_columns=['nt1', 'ntre'],
#     sort_by_date=True,
#     var_name='type',
#     value_name='date',
#     var_position=6,
#     value_position=7
# )

In [ ]:
# melt_intake

In [ ]:
# melt_intake.to_csv(r'soti_221001230930/intake_221001230930.csv')

In [ ]:
intake = pd.read_csv(r'soti_221001230930/intake_221001230930.csv')
intake['date'] = intake['date'].apply(pd.to_datetime)
intake[intake_numerical_variables] = intake[intake_numerical_variables].astype('Int64')
intake = intake[(intake['date'] > threshold_date)]

In [ ]:
intake

In [ ]:
# intake_assessment = intake[intake['type'] == 'ntre']
# intake_assessment[intake_assessment.duplicated(keep=False)]
# intake_assessment

In [ ]:
# Calling screening dataset to get some information for intake
scr = pd.read_csv(r'011/datasets/0000_ps_sc_092310.csv')
scr[screening_date_columns] = scr[screening_date_columns].apply(pd.to_datetime)
scr[screening_all_int_columns] = scr[screening_all_int_columns].astype('Int64')

In [ ]:
# scr

In [ ]:
unique_scr = scr.drop_duplicates(subset='rid')
intake_with_sex_and_age = pd.merge(intake, unique_scr[['rid','sex', 'age']], on='rid', how='left')

In [ ]:
intake_with_sex_and_age

In [ ]:
new_intake_with_sex_and_age = intake_with_sex_and_age[intake_with_sex_and_age['type'] == 'nt1']
new_intake_with_sex_and_age

In [ ]:
normalized_value_counts(new_intake_with_sex_and_age['sex'])

In [ ]:
ages = new_intake_with_sex_and_age['age']

In [ ]:
age_groups = categorize_and_count_ages(ages)

In [ ]:
normalized_value_counts(age_groups)

In [ ]:
# Counting intake over quarters
# first, I need to make the date column is the index
date_intake = intake.set_index('date')
quarterly_counts = date_intake.resample('Q').size()
quarterly_counts

In [ ]:
intake_victimhood = intake[victimhood]
counts = intake_victimhood.apply(lambda x: (x ==1).sum())
total_counts = intake_victimhood.count()
percentages = (counts / total_counts * 100).round(2)
counts_df = pd.DataFrame({
    'Column': counts.index,
    'Count of 1s': counts.values,
    'Total Observation': total_counts.values,
    'Percentage of 1s': percentages.values
})

counts_df